In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
import datetime
import os
import pickle

# get_test_feature
功能 根据-v和-station文件在"日期"目录下生成"线路-feature.txt"文件，记录测试集基本特征，并拼接测试数据

In [65]:
def find_history_dist(num,is_rain,is_workday,is_peek,on,start_station,next_station,time):
    with open('./history/'+str(num)+'_'+str(is_rain)+'_'+str(is_workday)+'_'+str(is_peek)+'.txt', 'rb') as data_file:
        b = pickle.load(data_file).reset_index()
#         b['O_TIME'] = pd.to_datetime(b['O_TIME'],format='%Y-%m-%d %H:%M:%S')
        b = b[(b['O_TIME']<time)]
        b = b.groupby(['O_UP','Source_Station','Target_Station']).mean().reset_index()
        # 	O_UP 	Source_Station 	Target_Station
        b = b[(b['O_UP']==on)&(b['Source_Station']==start_station)&(b['Target_Station']==next_station)]
        if b.shape[0] > 0:
            return b.iloc[0]
        else :
            dic = {'Distance':[-1],'Diff_Time':[-1]}
            x = pd.DataFrame(dic)
            return x.iloc[0]
        
def describe(x):
    if x=='多云':
        return 0
    elif x=='阴':
        return 1
    elif x=='晴':
        return 2
    elif x=='小雨':
        return 3
    else :
        return 4

def rain(d,n):
    if (int(d)>=3 or int(n)>=3):
        return 1
    else :
        return 0

weather = pd.read_csv("/home/b418-xiwei/xzl/notebook/DC/data/weather.txt", sep=",")
weather['days'] = pd.to_datetime(weather['days'],format='%Y年%m月%d日')
# weather[(weather['days']=='2017-10-'+str(train.iloc[1]['O_TIME'].day))]
# weather['dws'].drop_duplicates()
weather['dws_onehot'] = weather['dws'].apply(describe)
weather['nws_onehot'] = weather['nws'].apply(describe)
weather['is_rain'] = weather.apply(lambda x: rain(x['dws_onehot'], x['nws_onehot']), axis=1)


def get_featurea(test1):
    def read_station(line,on,start,end):
        path = './station/'+str(line)+'-dist.txt'
        if os.path.exists(path):
            bus1 = pd.read_table(path,header=None,sep=',',encoding='utf-8')
            bus1.columns=["O_UP","Source_Station","Target_Station","Distance"]
            b = bus1[(bus1['O_UP']==on)&(bus1['Source_Station']==start)&(bus1['Target_Station']==end)]
            if b.shape[0] > 0:
                return b.iloc[0]['Distance']
            else :
                return -1
        else :
            return -1
#     test1['prev_d'] = test1['distance']?
    test1['Distance'] = test1.apply(lambda x: read_station(x['LINE'],x['UP'], x['pred_start_stop_ID'], x['pred_end_stop_ID']), axis=1)
    #6 hour
    test1['hour'] = test1['realTime'].apply(lambda x: x.hour)
    # 7 is_peek
    test1['is_peek'] = test1['hour'].apply(lambda x: 1 if ((x>=7 and x <9) or (x>=16 and x<19)) else 0)

    # 8 weekday
    test1['weekday'] = test1['DATA'].dt.weekday

    # 9 is_workday
    test1['is_workday'] = test1['weekday'].apply(lambda x: 0 if x>=5 else 1)

    weat = weather[(weather['days'] == '2017-10-' + day)].iloc[0]
    # 10-14 dws nws dts nts is_rain
    test1['dws'] = weat['dws_onehot']
    test1['nws'] = weat['nws_onehot']
    test1['dts'] = weat['dts']
    test1['nts'] = weat['nts']
    test1['is_rain'] = weat['is_rain']

    # 15 max_v ?是否有用 存疑 test数据无法捕捉 置为-1 
    # def getV(day,line,time,term):  
    #     path1 = './' + day + '/' + str(line) + '-v.txt';
    #     if os.path.exists(path1):
    #         busmaxv = pd.read_table(path1, header=None, sep=',', encoding='utf-8')
    #         busmaxv.columns = ['O_TERMINALNO','O_TIME', "O_UP", "Source_Station", "Target_Station", "MaxV"]
    #         busmaxv['O_TIME'] = pd.to_datetime(busmaxv['O_TIME'],format='%Y-%m-%d %H:%M:%S')
    #         b = busmaxv[(busmaxv['O_TIME']<time)&(busmaxv['O_TERMINALNO']==term)]
    #         if b.shape[0] > 0:
    #             return b.iloc[-1]['MaxV']
    #         else:
    #             return -1
    # test1['max_v'] = test1.apply(lambda x:getV(day,x['LINE'],x['realTime'],x['TERMINALNO']),axis=1)
    test1['max_v'] = -1
    
    # history
    # 16 aver_diff
    tmp = test1.apply(lambda x: find_history_dist(x['LINE'],x['is_rain'], x['is_workday'],x['is_peek'],x['UP'], x['pred_start_stop_ID'], x['pred_end_stop_ID'],str(x['realTime'])), axis=1)
    test1['h_aver_diff'] = tmp['Diff_Time']
    # 17 aver_d
    test1['h_aver_d'] = tmp['Distance']
    # 18 aver_v
    def averV(d,n):
        if (int(d)==-1 or int(n)==-1):
    #         print(d,n)
            return -1
        else :
            return 3600 * d / n
    test1['h_aver_v'] = test1.apply(lambda x: averV(x['h_aver_d'],x['h_aver_diff']), axis=1)


    with open('./' + day + '/test-feature5.txt', 'wb') as data_file:
        pickle.dump(test1, data_file)

In [66]:
test = pd.read_csv("./toBePredicted_0607_segment.csv", sep=",")
test['DATA'] = pd.to_datetime('2017-'+test['DATA'],format='%Y-%m-%d %H:%M:%S')
test['realTime'] = pd.to_datetime(test['realTime'],format='%Y-%m-%d %H:%M:%S')

for d in range(25, 32):
    if d < 10:
        day = '0' + str(d)
    else:
        day = str(d)
    test1 = test[test['DATA']=='2017-10-'+day]
    print(test1.shape)
    get_featurea(test1)
    print(day,'finished')

(75342, 14)


/home/b418-xiwei/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/b418-xiwei/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/b418-xiwei/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

25 finished
(78993, 14)
26 finished
(79621, 14)
27 finished
(62504, 14)
28 finished
(61839, 14)
29 finished
(66951, 14)
30 finished
(65130, 14)
31 finished


In [67]:
#contact train
test_data = pd.DataFrame()
for d in range(25, 32):
    if d < 10:
        day = '0' + str(d)
    else:
        day = str(d)

    path = './'+day+'/test-feature5.txt'
    if os.path.exists(path):
        with open(path, 'rb') as data_file:
            data = pickle.load(data_file)
        test_data = pd.concat([test_data,data])
    print(day,"finished")
print(test_data.shape)
with open('./data/test5.txt', 'wb') as data_file:
    pickle.dump(test_data, data_file)

25 finished
26 finished
27 finished
28 finished
29 finished
30 finished
31 finished
(490380, 28)


# for test 

In [68]:
with open('./data/test5.txt', 'rb') as data_file:
    line = pickle.load(data_file)
line.columns

Index(['Unnamed: 0', 'DATA', 'LATITUDE', 'LINE', 'LONGITUDE', 'TERMINALNO',
       'UP', 'distance', 'lastStation_NO', 'lastStation_Time', 'predHour',
       'pred_end_stop_ID', 'pred_start_stop_ID', 'realTime', 'Distance',
       'hour', 'is_peek', 'weekday', 'is_workday', 'dws', 'nws', 'dts', 'nts',
       'is_rain', 'max_v', 'h_aver_diff', 'h_aver_d', 'h_aver_v'],
      dtype='object')

In [69]:
def idfeature(a,b,c,d):
    if c < 10:
        c = '0'+str(c)
    if d < 10:
        d = '0'+str(d)
    return str(a)+str(b)+str(c)+str(d)
line['ID'] = line.apply(lambda x: idfeature(x['LINE'],x['UP'],x['pred_start_stop_ID'],x['pred_end_stop_ID']),axis=1)
line = line[['ID','LINE', 'UP', 'pred_start_stop_ID', 'pred_end_stop_ID', 'Distance', 'distance',
       'realTime', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO']]
line

,ID,LINE,UP,pred_start_stop_ID,pred_end_stop_ID,Distance,distance,realTime,hour,is_peek,...,dws,nws,dts,nts,is_rain,max_v,h_aver_diff,h_aver_d,h_aver_v,TERMINALNO
209,86600809,866,0,8,9,0.629479,0.269546,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,104.605948,0.623653,21.462932,905578
210,86600910,866,0,9,10,0.517842,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,121.091912,0.520073,15.461508,905578
211,86601011,866,0,10,11,0.536539,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,116.723636,0.535451,16.514425,905578
212,86601112,866,0,11,12,0.776565,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,169.844203,0.770747,16.336670,905578
213,86601213,866,0,12,13,0.408048,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,71.837545,0.403532,20.222203,905578
214,86601314,866,0,13,14,0.983655,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,152.003663,0.979596,23.200388,905578
215,86601415,866,0,14,15,0.586326,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,116.649635,0.589106,18.180779,905578
216,86601516,866,0,15,16,0.480220,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,76.299639,0.476011,22.459331,905578
217,86601617,866,0,16,17,0.520341,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,196.512635,0.525341,9.623949,905578
218,86601718,866,0,17,18,0.794807,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,193.818182,0.787386,14.624997,905578


In [70]:
line.columns=['ID','O_LINENO', 'O_UP', 'Source_Station', 'Target_Station', 'Distance', 'distance',
       'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO']
line

,ID,O_LINENO,O_UP,Source_Station,Target_Station,Distance,distance,O_TIME,hour,is_peek,...,dws,nws,dts,nts,is_rain,max_v,h_aver_diff,h_aver_d,h_aver_v,TERMINALNO
209,86600809,866,0,8,9,0.629479,0.269546,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,104.605948,0.623653,21.462932,905578
210,86600910,866,0,9,10,0.517842,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,121.091912,0.520073,15.461508,905578
211,86601011,866,0,10,11,0.536539,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,116.723636,0.535451,16.514425,905578
212,86601112,866,0,11,12,0.776565,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,169.844203,0.770747,16.336670,905578
213,86601213,866,0,12,13,0.408048,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,71.837545,0.403532,20.222203,905578
214,86601314,866,0,13,14,0.983655,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,152.003663,0.979596,23.200388,905578
215,86601415,866,0,14,15,0.586326,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,116.649635,0.589106,18.180779,905578
216,86601516,866,0,15,16,0.480220,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,76.299639,0.476011,22.459331,905578
217,86601617,866,0,16,17,0.520341,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,196.512635,0.525341,9.623949,905578
218,86601718,866,0,17,18,0.794807,0.000000,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,193.818182,0.787386,14.624997,905578


In [71]:
a = []
for i in range(line.shape[0]):
    l = line.iloc[i]
    if l['distance'] > 10 or l['distance'] == -1:
        a.append(l['Distance'])
    elif l['Distance'] - l['distance'] < 0:
        a.append(0)
    else:
        a.append(l['Distance'] - l['distance'])
line['new_dist'] = a
line

,ID,O_LINENO,O_UP,Source_Station,Target_Station,Distance,distance,O_TIME,hour,is_peek,...,nws,dts,nts,is_rain,max_v,h_aver_diff,h_aver_d,h_aver_v,TERMINALNO,new_dist
209,86600809,866,0,8,9,0.629479,0.269546,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,104.605948,0.623653,21.462932,905578,0.359933
210,86600910,866,0,9,10,0.517842,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,121.091912,0.520073,15.461508,905578,0.517842
211,86601011,866,0,10,11,0.536539,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,116.723636,0.535451,16.514425,905578,0.536539
212,86601112,866,0,11,12,0.776565,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,169.844203,0.770747,16.336670,905578,0.776565
213,86601213,866,0,12,13,0.408048,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,71.837545,0.403532,20.222203,905578,0.408048
214,86601314,866,0,13,14,0.983655,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,152.003663,0.979596,23.200388,905578,0.983655
215,86601415,866,0,14,15,0.586326,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,116.649635,0.589106,18.180779,905578,0.586326
216,86601516,866,0,15,16,0.480220,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,76.299639,0.476011,22.459331,905578,0.480220
217,86601617,866,0,16,17,0.520341,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,196.512635,0.525341,9.623949,905578,0.520341
218,86601718,866,0,17,18,0.794807,0.000000,2017-10-25 09:00:00,9,0,...,1,17,10,0,-1,193.818182,0.787386,14.624997,905578,0.794807


In [72]:
line.columns
line.columns = ['ID', 'O_LINENO', 'O_UP', 'Source_Station', 'Target_Station',
       'Distance', 'distance2', 'O_TIME', 'hour', 'is_peek', 'weekday',
       'is_workday', 'dws', 'nws', 'dts', 'nts', 'is_rain', 'max_v',
       'h_aver_diff', 'h_aver_d', 'h_aver_v', 'TERMINALNO','new_dist']

with open('./data/test5-id.txt', 'wb') as data_file:
    pickle.dump(line, data_file)

In [18]:
test = pd.read_csv("./toBePredicted_531_segment.csv", sep=",")
test['DATA'] = pd.to_datetime('2017-'+test['DATA'],format='%Y-%m-%d %H:%M:%S')
test['realTime'] = pd.to_datetime(test['realTime'],format='%Y-%m-%d %H:%M:%S')
test

In [20]:
line[(line['Distance'] - line['distance']) < 0]

,ID,O_LINENO,O_UP,Source_Station,Target_Station,Distance,distance,O_TIME,hour,is_peek,...,dws,nws,dts,nts,is_rain,max_v,h_aver_diff,h_aver_d,h_aver_v,TERMINALNO
9817,68503233,685,0,32,33,2.287796,12329.893472,2017-10-25 11:00:00,11,0,...,1,1,17,10,0,-1,230.000000,2.293251,35.894362,902524
11521,73210304,732,1,3,4,0.578445,0.654540,2017-10-25 17:00:00,17,1,...,1,1,17,10,0,-1,141.452381,0.748020,19.037305,900552
13107,301011,3,0,10,11,0.592632,0.600239,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,126.513072,0.589634,16.778375,932508
13790,96100102,961,0,1,2,2.841063,24641.434837,2017-10-25 17:00:00,17,1,...,1,1,17,10,0,-1,581.223810,2.828353,17.518330,903088
15606,84002930,840,0,29,30,0.722551,0.736947,2017-10-25 11:00:00,11,0,...,1,1,17,10,0,-1,272.089686,0.710573,9.401548,903115
21400,90613334,906,1,33,34,1.024778,1.510207,2017-10-25 11:00:00,11,0,...,1,1,17,10,0,-1,165.132184,1.026038,22.368365,900887
25891,46201516,462,0,15,16,4.167935,14.844782,2017-10-25 09:00:00,9,0,...,1,1,17,10,0,-1,310.000000,4.359605,50.627674,901228
29119,90711718,907,1,17,18,1.213316,12321.459543,2017-10-25 19:00:00,19,0,...,1,1,17,10,0,-1,207.771186,1.199588,20.784968,905531
30885,58811920,588,1,19,20,8.784893,10.957189,2017-10-25 15:00:00,15,0,...,1,1,17,10,0,-1,-1.000000,-1.000000,-1.000000,904490
36841,83601213,836,0,12,13,0.603463,24641.388644,2017-10-25 07:00:00,7,1,...,1,1,17,10,0,-1,204.000000,0.653660,11.535173,905217


In [36]:
with open('./data/test4.txt', 'rb') as data_file:
    linex = pickle.load(data_file)
linex = linex[['DATA', 'LATITUDE', 'LINE', 'LONGITUDE', 'TERMINALNO',
       'UP', 'distance', 'lastStation_NO', 'lastStation_Time', 'predHour',
       'pred_end_stop_ID', 'pred_start_stop_ID', 'realTime', 'Distance',
       'hour', 'h_aver_diff', 'h_aver_d', 'h_aver_v']]
line = linex[(linex['Distance'] - linex['distance']) < 0]


In [31]:
day = '25'
train = pd.read_csv("/home/b418-xiwei/xzl/notebook/DC/data/train201710"+day+".csv", sep=",")
train['O_TIME'] = pd.to_datetime('2017-10-'+day+' '+train['O_TIME'],format='%Y-%m-%d %H:%M:%S')
train = train.groupby(['O_LINENO', 'O_TERMINALNO']).apply(lambda x:x.sort_values(by='O_TIME', ascending=True))
train = train[(train['O_TERMINALNO']==902524)]
bus = train[(train['O_TIME']>'2017-10-25 10:55:00')&(train['O_TIME']<'2017-10-25 11:00:00')]
bus

O_LINENO  O_TERMINALNO              O_TIME  \
O_LINENO O_TERMINALNO                                                       
685      902524       5402156       685        902524 2017-10-25 10:55:07   
                      5393710       685        902524 2017-10-25 10:55:17   
                      5398675       685        902524 2017-10-25 10:55:27   
                      5413719       685        902524 2017-10-25 10:55:37   
                      5410421       685        902524 2017-10-25 10:55:47   
                      5406277       685        902524 2017-10-25 10:55:57   
                      5415531       685        902524 2017-10-25 10:56:07   
                      5412212       685        902524 2017-10-25 10:56:17   
                      5408913       685        902524 2017-10-25 10:56:27   
                      5418972       685        902524 2017-10-25 10:56:37   
                      5415679       685        902524 2017-10-25 10:56:47   
                      5413193       685        902524 2017-10-25 10:56:58   
                      5409890       685        902524 2017-10-25 10:57:08   
                      5419158       685        902524 2017-10-25 10:57:18   
                      5415850       685        902524 2017-10-25 10:57:28   
                      5412547       685        902524 2017-10-25 10:57:38   
                      5409239       685        902524 2017-10-25 10:57:48   
                      5419294       685        902524 2017-10-25 10:57:58   
                      5415982       685        902524 2017-10-25 10:58:08   
                      5411855       685        902524 2017-10-25 10:58:17   
                      5408545       685        902524 2017-10-25 10:58:28   
                      5418604       685        902524 2017-10-25 10:58:37   
                      5415297       685        902524 2017-10-25 10:58:47   
                      5411193       685        902524 2017-10-25 10:58:57   
                      5408685       685        902524 2017-10-25 10:59:07   
                      5449181       685        902524 2017-10-25 10:59:18   
                      5448393       685        902524 2017-10-25 10:59:27   
                      5447454       685        902524 2017-10-25 10:59:37   
                      5446819       685        902524 2017-10-25 10:59:47   
                      5448921       685        902524 2017-10-25 10:59:57   

                               O_LONGITUDE  O_LATITUDE  O_SPEED  O_MIDDOOR  \
O_LINENO O_TERMINALNO                                                        
685      902524       5402156      117.261     39.0852       40          1   
                      5393710      117.262     39.0858       29          1   
                      5398675      117.262     39.0858        0          1   
                      5413719      117.262     39.0858        0          1   
                      5410421        0.000      0.0000        0          1   
                      5406277      117.262     39.0862       35          1   
                      5415531      117.264     39.0869       44          1   
                      5412212      117.265     39.0875       38          1   
                      5408913      117.267     39.0879       37          1   
                      5418972      117.268     39.0886       33          1   
                      5415679      117.269     39.0895       48          1   
                      5413193      117.270     39.0906       42          1   
                      5409890      117.270     39.0910        5          1   
                      5419158      117.270     39.0910        0          1   
                      5415850      117.271     39.0910       14          1   
                      5412547      117.272     39.0914       24          1   
                      5409239      117.272     39.0921       37          1   
                      5419294      117.273     39.0929       38          1   
      

In [64]:


#d2因为没去0经纬度的有420个偏大
#linex[linex['distance'] >10] -> d1
#小于0 -> 0
#其他d1-d2

In [43]:
a = []
for i in range(linex.shape[0]):
    l = linex.iloc[i]
    if l['distance'] > 10 or l['distance'] == -1:
        a.append(l['Distance'])
    elif l['Distance'] - l['distance'] < 0:
        a.append(0)
    else:
        a.append(l['Distance'] - l['distance'])
linex['new_dist'] = a
linex

,DATA,LATITUDE,LINE,LONGITUDE,TERMINALNO,UP,distance,lastStation_NO,lastStation_Time,predHour,pred_end_stop_ID,pred_start_stop_ID,realTime,Distance,hour,h_aver_diff,h_aver_d,h_aver_v,a,new_dist
209,2017-10-25,39.0848,866,117.179,905578,0,0.269546,8,2017-10-25 08:58:57,9:00:00,9,8,2017-10-25 09:00:00,0.629479,9,104.605948,0.623653,21.462932,0.359933,0.359933
210,2017-10-25,39.0848,866,117.179,905578,0,0.000000,8,2017-10-25 08:58:57,9:00:00,10,9,2017-10-25 09:00:00,0.517842,9,121.091912,0.520073,15.461508,0.517842,0.517842
211,2017-10-25,39.0848,866,117.179,905578,0,0.000000,8,2017-10-25 08:58:57,9:00:00,11,10,2017-10-25 09:00:00,0.536539,9,116.723636,0.535451,16.514425,0.536539,0.536539
212,2017-10-25,39.0848,866,117.179,905578,0,0.000000,8,2017-10-25 08:58:57,9:00:00,12,11,2017-10-25 09:00:00,0.776565,9,169.844203,0.770747,16.336670,0.776565,0.776565
213,2017-10-25,39.0848,866,117.179,905578,0,0.000000,8,2017-10-25 08:58:57,9:00:00,13,12,2017-10-25 09:00:00,0.408048,9,71.837545,0.403532,20.222203,0.408048,0.408048
214,2017-10-25,39.0848,866,117.179,905578,0,0.000000,8,2017-10-25 08:58:57,9:00:00,14,13,2017-10-25 09:00:00,0.983655,9,152.003663,0.979596,23.200388,0.983655,0.983655
215,2017-10-25,39.0848,866,117.179,905578,0,0.000000,8,2017-10-25 08:58:57,9:00:00,15,14,2017-10-25 09:00:00,0.586326,9,116.649635,0.589106,18.180779,0.586326,0.586326
216,2017-10-25,39.0848,866,117.179,905578,0,0.000000,8,2017-10-25 08:58:57,9:00:00,16,15,2017-10-25 09:00:00,0.480220,9,76.299639,0.476011,22.459331,0.480220,0.480220
217,2017-10-25,39.0848,866,117.179,905578,0,0.000000,8,2017-10-25 08:58:57,9:00:00,17,16,2017-10-25 09:00:00,0.520341,9,196.512635,0.525341,9.623949,0.520341,0.520341
218,2017-10-25,39.0848,866,117.179,905578,0,0.000000,8,2017-10-25 08:58:57,9:00:00,18,17,2017-10-25 09:00:00,0.794807,9,193.818182,0.787386,14.624997,0.794807,0.794807


In [47]:
linex.columns

Index(['DATA', 'LATITUDE', 'LINE', 'LONGITUDE', 'TERMINALNO', 'UP', 'distance',
       'lastStation_NO', 'lastStation_Time', 'predHour', 'pred_end_stop_ID',
       'pred_start_stop_ID', 'realTime', 'Distance', 'hour', 'h_aver_diff',
       'h_aver_d', 'h_aver_v', 'a', 'new_dist'],
      dtype='object')